In [10]:
import pandas as pd

ksi_df = pd.read_csv("data/KSI.csv")  # Adjust filename as needed

In [11]:
import geopandas as gpd

ksi_gdf = gpd.read_file("data/KSI.geojson")  # or .shp

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

# Drop missing coordinates
ksi_df = ksi_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Combine into geometry
geometry = [Point(xy) for xy in zip(ksi_df['LONGITUDE'], ksi_df['LATITUDE'])]
ksi_gdf = gpd.GeoDataFrame(ksi_df, geometry=geometry, crs='EPSG:4326')

ModuleNotFoundError: No module named 'shapely'

In [ ]:
# Strip column names
ksi_gdf.columns = ksi_gdf.columns.str.strip()

# Convert date/time
# Convert TIME to string format (HHMM -> HH:MM)
ksi_gdf['TIME_STR'] = ksi_gdf['TIME'].astype(str).str.zfill(4).str[:2] + ':' + ksi_gdf['TIME'].astype(str).str.zfill(4).str[2:]
ksi_gdf['DATETIME'] = pd.to_datetime(ksi_gdf['DATE'] + ' ' + ksi_gdf['TIME_STR'], errors='coerce')
ksi_gdf['hour'] = ksi_gdf['DATETIME'].dt.hour
ksi_gdf['weekday'] = ksi_gdf['DATETIME'].dt.dayofweek

# Encode severity
ksi_gdf['severity_score'] = 5  # All entries are Killed or Seriously Injured

# Optional: simplify or encode weather/light conditions if present
if 'LIGHT' in ksi_gdf.columns:
    ksi_gdf['light_code'] = ksi_gdf['LIGHT'].astype('category').cat.codes

if 'WEATHER' in ksi_gdf.columns:
    ksi_gdf['weather_code'] = ksi_gdf['WEATHER'].astype('category').cat.codes